In [1]:
# Attention !
# if visualization in dune is not working then
# > pip uninstall k3d
# > pip install K3D==2.6.6

In [2]:
# wurlitzer: display dune's output in the notebook
# %load_ext wurlitzer

%matplotlib inline

import numpy as np
np.warnings.filterwarnings('ignore') # silence numpys warnings

In [3]:
from pymor.core.logger import set_log_levels
set_log_levels({'pymor': 'WARN'})

In [4]:
# export the thermal-block problem as a pymor problem
from pymor.analyticalproblems.thermalblock import thermal_block_problem
from pymor.analyticalproblems.domaindescriptions import RectDomain

d = 2
omega = ([0, 0], [1, 1])
blocks = (2, 2)
# blocks = (1, 1)
problem = thermal_block_problem(blocks)

In [5]:
# add parametric rhs

from pymor.analyticalproblems.functions import LincombFunction, ConstantFunction
from pymor.parameters.functionals import ProjectionParameterFunctional

param_rhs = LincombFunction([ConstantFunction(0.5, 2), ConstantFunction(0.5, 2)],
                            [ProjectionParameterFunctional('rhs', 2, 0),
                             ProjectionParameterFunctional('rhs', 2, 1)])

problem = problem.with_(rhs = param_rhs)

In [6]:
# the standard pymor way
from pymor.discretizers.builtin.grids.rect import RectGrid
from pymor.discretizers.builtin.cg import discretize_stationary_cg
from pymor.discretizers.builtin.fv import discretize_stationary_fv

h = 0.01
pymor_cg, data = discretize_stationary_cg(problem, diameter=h, grid_type=RectGrid)
pymor_fv, data_ = discretize_stationary_fv(problem, diameter=h, grid_type=RectGrid)

In [7]:
mu = [1., 0.1, 1., 0.1, 1., 1.]
# mu = [1.]

In [8]:
pymor_grid = data['grid']
print(pymor_grid)

Rect-Grid on domain [0,1] x [0,1]
x0-intervals: 142, x1-intervals: 142
elements: 20164, edges: 40612, vertices: 20449


In [9]:
# solving the problem

u_pymor = pymor_cg.solve(mu)
pymor_cg.visualize(u_pymor)

Output()

In [10]:
# solving the problem with dune cg
from pymor.discretizers.dunegdt.cg import discretize_stationary_cg as discretize_dune_cg

pymor_dune_cg, _ = discretize_dune_cg(problem, diameter=h)

u_dune_cg = pymor_dune_cg.solve(mu)
_ = pymor_dune_cg.visualize(u_dune_cg)

Output()

In [11]:
# solving the problem with dune ipdg
from pymor.discretizers.dunegdt.ipdg import discretize_stationary_ipdg as discretize_dune_ipdg

pymor_dune_ipdg, _ = discretize_dune_ipdg(problem, diameter=h)

u_dune_ipdg = pymor_dune_ipdg.solve(mu)
_ = pymor_dune_ipdg.visualize(u_dune_ipdg)

Output()

# IPL Using DD Glued

In [12]:
from pymor.discretizers.dunegdt.ipld3g import discretize_stationary_ipld3g

In [13]:
N = 4.
macro_diameter = np.sqrt(2)/N

pymor_ipl_model, data = discretize_stationary_ipld3g(problem, macro_diameter=macro_diameter,
                                                     num_local_refinements=2,
                                                     penalty_parameter=16.)

In [14]:
u_pymor_ipl_dune = pymor_ipl_model.solve(mu)

In [15]:
macro_grid = data['macro_grid']
dd_grid = data['dd_grid']
local_spaces = data['local_spaces']

data.keys()

dict_keys(['macro_grid', 'dd_grid', 'macro_boundary_info', 'local_spaces', 'coupling_op', 'lhs_without_coupling', 'unassembled_m'])

In [16]:
# visualization
from dd_glued_visualizer import visualize_dd_functions

visualize_dd_functions(dd_grid, local_spaces, u_pymor_ipl_dune)

ViewInteractiveWidget(height=1100, layout=Layout(height='auto', width='100%'), width=1500)

In [17]:
from dune.xt.grid import visualize_grid
_ = visualize_grid(macro_grid)

Output()

## Enrichment locally with global solutions

In [18]:
us = pymor_ipl_model.solution_space.empty()

for mu_ in problem.parameter_space.sample_randomly(5):
    print('.', end='', flush=True)
    us.append(pymor_ipl_model.solve(mu_))

.....

In [19]:
from pymor.reductors.coercive_ipl3dg import CoerciveIPLD3GRBReductor

localized_reductor = CoerciveIPLD3GRBReductor(pymor_ipl_model, dd_grid)

In [20]:
print(localized_reductor.basis_length())
localized_reductor.add_global_solutions(us[:3])
print(localized_reductor.basis_length())

for i in range(0, localized_reductor.S):
    u = us[3:].block(i)
    localized_reductor.add_local_solutions(i, u)

print(localized_reductor.basis_length())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [21]:
localized_rom = localized_reductor.reduce()

u_rom_loc = localized_rom.solve(mu)

u_rom_loc_reconstructed = localized_reductor.reconstruct(u_rom_loc)

In [22]:
visualize_dd_functions(dd_grid, local_spaces, u_rom_loc_reconstructed)

ViewInteractiveWidget(height=1100, layout=Layout(height='auto', width='100%'), width=1500)

## Estimation

### estimator domains

In [23]:
localized_reductor.element_patches

[(0, 1, 4, 5),
 (0, 1, 2, 4, 5, 6),
 (1, 2, 3, 5, 6, 7),
 (2, 3, 6, 7),
 (0, 1, 4, 5, 8, 9),
 (0, 1, 2, 4, 5, 6, 8, 9, 10),
 (1, 2, 3, 5, 6, 7, 9, 10, 11),
 (2, 3, 6, 7, 10, 11),
 (4, 5, 8, 9, 12, 13),
 (4, 5, 6, 8, 9, 10, 12, 13, 14),
 (5, 6, 7, 9, 10, 11, 13, 14, 15),
 (6, 7, 10, 11, 14, 15),
 (8, 9, 12, 13),
 (8, 9, 10, 12, 13, 14),
 (9, 10, 11, 13, 14, 15),
 (10, 11, 14, 15)]

In [24]:
localized_reductor.estimator_domains

{0: [0, 1, 2, 4, 5, 6, 8, 9],
 1: [0, 1, 2, 3, 4, 5, 6, 7, 9, 10],
 2: [1, 2, 3, 5, 6, 7, 10, 11],
 4: [0, 1, 4, 5, 6, 8, 9, 10, 12, 13],
 5: [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14],
 6: [2, 3, 5, 6, 7, 9, 10, 11, 14, 15],
 8: [4, 5, 8, 9, 10, 12, 13, 14],
 9: [5, 6, 8, 9, 10, 11, 12, 13, 14, 15],
 10: [6, 7, 9, 10, 11, 13, 14, 15]}